In [4]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Real-time Top Selling Products

## Description of the use case
This use case involves keeping an updated list of top-selling products/services in real-time. By tracking sales data, banks can identify and promote their best-selling products.

## Redis Data Structures Used
- **Top-K**: To maintain a list of top-selling products.
- **Hashes**: To store sales data for each product.


In [5]:
r.flushdb()
# Mock data for the use case
sales_data = {
    'order_1001': {'name': 'Savings Account', 'sales': 100},
    'order_1002': {'name': 'Overdraft Account', 'sales': 150},
    'order_1003': {'name': 'Fixed Deposit (Short)', 'sales': 200},
    'order_1004': {'name': 'Fixed Deposit (Long)', 'sales': 50},
    'order_1005': {'name': 'Home Loan', 'sales': 300},
    'order_1006': {'name': 'Savings Account', 'sales': 70},
    'order_1007': {'name': 'Savings Account', 'sales': 55},
    'order_1008': {'name': 'Fixed Deposit (Short)', 'sales': 500},
    'order_1009': {'name': 'Overdraft Account', 'sales': 6},
    'order_1010': {'name': 'Home Loan', 'sales': 200}
}

# Use Hashes to store sales data for each product
for product_id, data in sales_data.items():
    r.hset(product_id, mapping=data)

# Initialize Top-K data structure
r.topk().reserve('top_selling_products', k=3, width=50, depth=7,decay=0.9)

# Function to update Top-K with sales data
def update_top_selling_products():
    for product_id, data in sales_data.items():
        r.topk().incrby('top_selling_products',[data['name']],[data['sales']])

# Update Top-K with the sales data
update_top_selling_products()

# Print the sales data for each product
for product_id in sales_data.keys():
    print(f"{product_id}: {r.hgetall(product_id)}")

# Get the list of top-selling products
top_selling_products = r.topk().list('top_selling_products', withcount=True)


print('Top Selling Products:')
for i in range(0,len(top_selling_products),2):
    print(f"Product ({top_selling_products[i]}) : {top_selling_products[i+1]}")

order_1001: {'name': 'Savings Account', 'sales': '100'}
order_1002: {'name': 'Overdraft Account', 'sales': '150'}
order_1003: {'name': 'Fixed Deposit (Short)', 'sales': '200'}
order_1004: {'name': 'Fixed Deposit (Long)', 'sales': '50'}
order_1005: {'name': 'Home Loan', 'sales': '300'}
order_1006: {'name': 'Savings Account', 'sales': '70'}
order_1007: {'name': 'Savings Account', 'sales': '55'}
order_1008: {'name': 'Fixed Deposit (Short)', 'sales': '500'}
order_1009: {'name': 'Overdraft Account', 'sales': '6'}
order_1010: {'name': 'Home Loan', 'sales': '200'}
Top Selling Products:
Product (Fixed Deposit (Short)) : 700
Product (Home Loan) : 500
Product (Savings Account) : 225
